<a href="https://colab.research.google.com/github/TsaiMandy9112/Artificial-Intelligence/blob/main/AI_Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SimpleRNN**

In [ ]:
import tensorflow as tf
tf.random.set_seed(1)

# https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN
# units: positive integer, dimensionality of the output space
# return_sequences: Boolean.
#   Whether to return the last output in the output sequence, or the full sequence.
#   Default: False.
rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True, activation='tanh', dropout=0,
    return_sequences=True)

# inputs: A 3D tensor, with shape [batch, timesteps, feature]
rnn_layer.build(input_shape=(None, None, 5))

w_xh, w_oo, b_h = rnn_layer.weights

print('W_xh shape:', w_xh.shape)
print('W_oo shape:', w_oo.shape)
print('b_h shape:', b_h.shape)

W_xh shape: (5, 2)
W_oo shape: (2, 2)
b_h shape: (2,)


# **Manually computering the output**

In [ ]:
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)
print(x_seq)

tf.Tensor(
[[1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3.]], shape=(3, 5), dtype=float32)


In [ ]:
# Convert x_seq to tensor structure
# x_seq shape: (3, 5)
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5],
    dtype=tf.float32)

# output of SimepleRNN:
# [batch=1, timesteps=3, feature=5]
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(len(x_seq)):
  # x_seq[t] shape: (5,) need reshape to (1, 5)
  xt = tf.reshape(x_seq[t], (1, 5))
  print('Time step {} =>'.format(t))
  print('  Input:', xt.numpy())

  ht = tf.matmul(xt, w_xh) + b_h
  print('  w_xh:',w_xh)
  print('  Hidden:', ht.numpy())

  if t > 0:
    prev_o = out_man[t-1]
  else:
    prev_o = tf.zeros(shape=(ht.shape))

  ot = ht + tf.matmul(prev_o, w_oo)
  print('  w_oo:',w_oo)
  print('  ot:',ot)

  ot = tf.math.tanh(ot)
  out_man.append(ot)
  print('  Output(manual):', ot.numpy())
  print('  SimpleRNN output:'.format(t), output[0][t].numpy())
  print('=======================================')

Time step 0 =>
  Input: [[1. 1. 1. 1. 1.]]
  w_xh: <tf.Variable 'simple_rnn_cell_12/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[-0.6200572 ,  0.7433989 ],
       [ 0.242517  , -0.12119704],
       [-0.38525409,  0.2638626 ],
       [ 0.8809836 , -0.12017238],
       [ 0.2964511 ,  0.19422936]], dtype=float32)>
  Hidden: [[0.41464037 0.96012145]]
  w_oo: <tf.Variable 'simple_rnn_cell_12/recurrent_kernel:0' shape=(2, 2) dtype=float32, numpy=
array([[ 0.98796964,  0.15464693],
       [-0.15464693,  0.9879698 ]], dtype=float32)>
  ot: tf.Tensor([[0.41464037 0.96012145]], shape=(1, 2), dtype=float32)
  Output(manual): [[0.39240566 0.74433106]]
  SimpleRNN output: [0.39240566 0.74433106]
Time step 1 =>
  Input: [[2. 2. 2. 2. 2.]]
  w_xh: <tf.Variable 'simple_rnn_cell_12/kernel:0' shape=(5, 2) dtype=float32, numpy=
array([[-0.6200572 ,  0.7433989 ],
       [ 0.242517  , -0.12119704],
       [-0.38525409,  0.2638626 ],
       [ 0.8809836 , -0.12017238],
       [ 0.2964511 ,  0.1942293

# Implementing RNNs for sequence modeling in TensorFlow

In [ ]:
import gdown

data_path = 'https://drive.google.com/u/0/uc?id=1nD_ZvxO7d_VusafIwrPY_AZeFLU0s39e&export=download'
gdown.download(data_path, 'movie_data.csv.gz')
print('finish')

Downloading...
From: https://drive.google.com/u/0/uc?id=1nD_ZvxO7d_VusafIwrPY_AZeFLU0s39e&export=download
To: /content/movie_data.csv.gz
100%|██████████| 26.5M/26.5M [00:01<00:00, 23.4MB/s]

finish


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
import gzip
import shutil

fname = 'movie_data.csv.gz'

# Read IMDb Dataset
with gzip.open(fname, 'rb') as f_in, open('movie_data.csv', 'wb') as f_out:
  shutil.copyfileobj(f_in, f_out)

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


In [ ]:
# Step 1: Create a dataset
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))

# inspection(檢查):
# take(3)：取 3 次
for ex in ds_raw.take(3):
  tf.print(ex[0].numpy()[0][:100], ex[1])

(b'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class a'
 b'rea of Belle Haven, Gree') 1
(b'OK... so... I really like Kris Kristofferson and his usual easy going delive'
 b'ry of lines in his movie') 0
(b'***SPOILER*** Do not read this, if you think about watching that movie, alth'
 b'ough it would be a waste') 0


In [ ]:
# Train/validaiton/test splits
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

print('ds_raw_test:',len(ds_raw_test))
print('ds_raw_train_valid:',len(ds_raw_train_valid))
print('ds_raw_train:',len(ds_raw_train))
print('ds_raw_valid:',len(ds_raw_valid))

ds_raw_test: 25000
ds_raw_train_valid: 25000
ds_raw_train: 20000
ds_raw_valid: 5000


In [ ]:
# Tokenizer and Encoder
# tfds.features.text.Tokenizer: https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/Tokenizer
# tfds.features.text.TokenTextEncoder: https://www.tensorflow.org/datasets/api_docs/python/tfds/features/text/TokenTextEncoder
# Encoding sequences: keeping the last 100 items in each sequence

# Step 2: find unique tokens (words)
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
  tokens = tokenizer.tokenize(example[0].numpy()[0])
  token_counts.update(tokens)

print('Vocab-size:', len(token_counts))


Vocab-size: 87007


In [ ]:
# Step 3: encoding each unique token into integers
# oov_buckets: int, the number of ints to reserve for OOV hash buckets. Tokens that are OOV will be hash-modded into a OOV bucket in encode.
# oov_token: str, the string to use for OOV ids in decode.
# strip_vocab: bool, whether to strip whitespace from the beginning and end of elements of vocab_list.
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts, oov_buckets=1,
    oov_token='UNK', lowercase=False,
    tokenizer=None, strip_vocab=True, decode_token_separator=' ')

example_str = 'This is an example!'
encoder.encode(example_str)

[232, 9, 270, 1123]

In [ ]:
# Step 3-A: define the function for transformation
def encode(text_tensor, label):
  text = text_tensor.numpy()[0]
  encoded_text = encoder.encode(text)
  return encoded_text, label

## Step 3-B: wrap the encode function to a TF Op.
# 使用 tf.py_function 將 encode 函數包裝起來，將它轉換為 tensorflow 運算子
# 將影評文本編碼為整數串列的過程
def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label],
             Tout=(tf.int64, tf.int64))

ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

tf.random.set_seed(1)
# 顯示目前序列長度不同
for example in ds_train.shuffle(1000).take(5):
  print('Sequence length:', example[0].shape)

Sequence length: (24,)
Sequence length: (179,)
Sequence length: (262,)
Sequence length: (535,)
Sequence length: (130,)


In [ ]:
# Take a small subset
# 取出大小為 8 的子集
ds_subset = ds_train.take(8)
for example in ds_subset:
  print('Individual size:', example[0].shape)

## batching the datasets
# 設定 batch size = 4
ds_batched = ds_subset.padded_batch(
  4, padded_shapes=([-1], []))

for batch in ds_batched:
  print('Batch dimension:', batch[0].shape)


Individual size: (119,)
Individual size: (688,)
Individual size: (308,)
Individual size: (204,)
Individual size: (326,)
Individual size: (240,)
Individual size: (127,)
Individual size: (453,)
Batch dimension: (4, 688)
Batch dimension: (4, 453)


In [ ]:
# batching the datasets
train_data = ds_train.padded_batch(
    32, padded_shapes=([-1],[]))

valid_data = ds_valid.padded_batch(
    32, padded_shapes=([-1],[]))

test_data = ds_test.padded_batch(
    32, padded_shapes=([-1],[]))

In [ ]:
# Embedding layers for sentence encoding
from tensorflow.keras.layers import Embedding

model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
          output_dim=6,
          input_length=20,
          name='embed-layer'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding

In [ ]:
# An example of building a RNN model
# with SimpleRNN layer
from tensorflow.keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 324,193
Trainable params: 324,193
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# An example of building a RNN model
# with LSTM layer
from tensorflow.keras.layers import LSTM

model = Sequential()
model.add(Embedding(10000, 32))
model.add(LSTM(32, return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, None, 32)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 336,673
Trainable params: 336,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# An example of building a RNN model
# with GRU layer
from tensorflow.keras.layers import GRU

model = Sequential()
model.add(Embedding(10000, 32))
model.add(GRU(32, return_sequences=True))
model.add(GRU(32))
model.add(Dense(1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 32)          320000    
                                                                 
 gru_2 (GRU)                 (None, None, 32)          6336      
                                                                 
 gru_3 (GRU)                 (None, 32)                6336      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 332,705
Trainable params: 332,705
Non-trainable params: 0
_________________________________________________________________


In [ ]:
embedding_dim = 32
vocab_size = len(token_counts) + 2
tf.random.set_seed(1)

# build the model
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),

    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 32)          2784288   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               49664     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,842,273
Trainable params: 2,842,273
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# compile and train:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10)

# evaluate on the test data
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

Epoch 1/10
625/625 [==============================] - 386s 612ms/step - loss: 0.4895 - accuracy: 0.7603 - val_loss: 0.4793 - val_accuracy: 0.7728
Epoch 2/10
625/625 [==============================] - 374s 599ms/step - loss: 0.2631 - accuracy: 0.9004 - val_loss: 0.4225 - val_accuracy: 0.8600


In [ ]:
if not os.path.exists('models'):
    os.mkdir('models')

bi_lstm_model.save('Bidir-LSTM-full-length-seq.h5')

# Trying SimpleRNN with short sequences

In [ ]:
# Trying SimpleRNN with short sequences
def preprocess_datasets(
  ds_raw_train,
  ds_raw_valid,
  ds_raw_test,
  max_seq_length=None,
  batch_size=32):

  # Step 1: (already done => creating a dataset)
  # Step 2: find unique tokens
  tokenizer = tfds.deprecated.text.Tokenizer()
  token_counts = Counter()

  for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    if max_seq_length is not None:
      tokens = tokens[-max_seq_length:]
    token_counts.update(tokens)

  print('Vocab-size:', len(token_counts))

  # Step 3: encoding the texts
  encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
  def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    if max_seq_length is not None:
      encoded_text = encoded_text[-max_seq_length:]
    return encoded_text, label

  def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                Tout=(tf.int64, tf.int64))

  ds_train = ds_raw_train.map(encode_map_fn)
  ds_valid = ds_raw_valid.map(encode_map_fn)
  ds_test = ds_raw_test.map(encode_map_fn)

  # Step 4: batching the datasets
  train_data = ds_train.padded_batch(
    batch_size, padded_shapes=([-1],[]))

  valid_data = ds_valid.padded_batch(
    batch_size, padded_shapes=([-1],[]))

  test_data = ds_test.padded_batch(
    batch_size, padded_shapes=([-1],[]))

  return (train_data, valid_data,
      test_data, len(token_counts))

In [ ]:
def build_rnn_model(embedding_dim, vocab_size,
          recurrent_type='SimpleRNN',
          n_recurrent_units=64,
          n_recurrent_layers=1,
          bidirectional=True):

  tf.random.set_seed(1)
  # build the model
  model = tf.keras.Sequential()
  model.add(
    Embedding(input_dim=vocab_size,
         output_dim=embedding_dim,
         name='embed-layer')
    )

  for i in range(n_recurrent_layers):
    return_sequences = (i < n_recurrent_layers-1)

    if recurrent_type == 'SimpleRNN':
      recurrent_layer = SimpleRNN(
            units=n_recurrent_units,
            return_sequences=return_sequences,
            name='simprnn-layer-{}'.format(i))
    elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
            units=n_recurrent_units,
            return_sequences=return_sequences,
            name='lstm-layer-{}'.format(i))
    elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
            units=n_recurrent_units,
            return_sequences=return_sequences,
            name='gru-layer-{}'.format(i))

    if bidirectional:
        recurrent_layer = Bidirectional(
          recurrent_layer, name='bidir-'+recurrent_layer.name)

    model.add(recurrent_layer)
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

  return model

In [ ]:
from tensorflow.keras.layers import Bidirectional

batch_size = 32
embedding_dim = 20
max_seq_length = 100

train_data, valid_data, test_data, n = preprocess_datasets(
  ds_raw_train, ds_raw_valid, ds_raw_test,
  max_seq_length=max_seq_length,
  batch_size=batch_size
)

# index = 0 for padding placeholder
# index = n + 1 for oov
vocab_size = n + 2

rnn_model = build_rnn_model(
  embedding_dim, vocab_size,
  recurrent_type='SimpleRNN',
  n_recurrent_units=64,
  n_recurrent_layers=1,
  bidirectional=True)

rnn_model.summary()

In [ ]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
         loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
         metrics=['accuracy'])

history = rnn_model.fit(train_data, validation_data=valid_data, epochs=10)

In [ ]:
results = rnn_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(results[1]*100))

# 利用Keras建構LSTM模型，以Stock Prediction 為例

In [ ]:
# 來源：https://reurl.cc/OXaEE7
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 連接 google 雲端硬碟
from google.colab import drive
drive.mount('/content/drive')
fname = '/content/drive/MyDrive/AI/SPY.csv'
# 連接 google 雲端硬碟

def readTrain():
  train = pd.read_csv(fname)
  return train

# 切割時間，增加Features，例如星期幾、幾月、幾號等。
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

# 將所有資料做正規化，而由於Date是字串非數字，因此先將它drop掉
def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  return train_norm

# Build Training Data
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

# 打亂順序
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

# Training data & Validation data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

# 模型建置
# 一對一
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model
# 多對一
def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model
# 一對多
def buildOneToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
  model.add(Dense(1))
  model.add(RepeatVector(5))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model
# 多對多
def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model


# 一對一訓練過程
# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 1 days to predict next 1 days
X_train, Y_train = buildTrain(train_norm, 1, 1)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
# X_trian: (5710, 30, 10)
# Y_train: (5710, 5, 1)
# X_val: (634, 30, 10)
# Y_val: (634, 5, 1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

# 多對一訓練過程
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

# 一對多訓練過程
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

# 多對多訓練模型
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


# Bidirectional LSTM on IMDB

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))


# Character-level text generation with LSTM

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

# Prepare the data
path = keras.utils.get_file(
    "nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt"
)
with io.open(path, encoding="utf-8") as f:
  text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i : i + maxlen])
  next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

# Build the model: a single LSTM layer
model = keras.Sequential(
  [
    keras.Input(shape=(maxlen, len(chars))),
    layers.LSTM(128),
    layers.Dense(len(chars), activation="softmax"),
  ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

# Prepare the text sampling function
def sample(preds, temperature=1.0):
  # helper function to sample an index from a probability array
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

# Train the model
epochs = 40
batch_size = 128

for epoch in range(epochs):
  model.fit(x, y, batch_size=batch_size, epochs=1)
  print()
  print("Generating text after epoch: %d" % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    print("...Diversity:", diversity)

    generated = ""
    sentence = text[start_index : start_index + maxlen]
    print('...Generating with seed: "' + sentence + '"')

    for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char

  print("...Generated: ", generated)
  print()


606208/600901 [==============================] - 0s 0us/step
Corpus length: 600893
Total chars: 56
Number of sequences: 200285
1565/1565 [==============================] - 187s 118ms/step - loss: 2.2202

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "od psychology served not only to cast su"
...Diversity: 0.5
...Generating with seed: "od psychology served not only to cast su"
...Diversity: 1.0
...Generating with seed: "od psychology served not only to cast su"
...Diversity: 1.2
...Generating with seed: "od psychology served not only to cast su"
...Generated:  co"--more mostanch so thin, thought and ever the impaan digredting, rowly, the grad at such mist!" which muy we newly be over haivice" does detainery accourengs quite;--nauss phich go "tone serensiness themen but so faiturlidingated ateal war defect light knowlecs belombois.  180. advate?, only moreer, with hards spirites stoyed, "bach the lems, antay as" oven _ emeepces! us--beporent gor-meinnt l

1565